In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import pandas as pd

In [2]:
def initialize_weights(row,col):   
    return tf.Variable(tf.random.uniform(shape=(row,col),maxval=0.4,minval=0)),\
            tf.Variable(tf.random.uniform(shape=(col,1),maxval=0,minval=0))


In [3]:
def activation_relu(z):
    zeros = tf.zeros_like(z)
    return tf.maximum(z,zeros)

In [4]:
def sigmoid(z):
    return (tf.Variable(1.0) / (tf.Variable(1.0) + tf.math.exp(-z)))

In [5]:
def phi(W,b,X):
    return (tf.transpose(W) @ X) + b

In [95]:
def soft_max(z):
    z = tf.transpose(z)
    s = tf.math.reduce_max(z, axis=1)
    s = tf.expand_dims(s,axis=1) # necessary step to do broadcasting
    e_x = tf.math.exp(z - s)
    div = tf.reduce_sum(e_x, axis=1)
    div = tf.expand_dims(div,axis=1) # dito
    output = tf.transpose(e_x / div)
    return output


In [94]:
def forward_propagation(weight,X,y):
    out1 = activation_relu(phi(weight['wl1'],weight['bl1'],X))
    out2 = activation_relu(phi(weight['wl2'],weight['bl2'],out1))
    outf = soft_max(phi(weight['wl3'],weight['bl3'],out2))
    cost = tf.reduce_mean((y-outf)**2)
    # cost = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(tf.transpose(y),tf.transpose(outf)))
    return cost,outf

In [93]:
def back(weight,X,y):
    with tf.GradientTape() as tape:
        out1 = activation_relu(phi(weight['wl1'],weight['bl1'],X))
        out2 = activation_relu(phi(weight['wl2'],weight['bl2'],out1))
        outf = soft_max(phi(weight['wl3'],weight['bl3'],out2))
        cost = tf.reduce_mean((y-outf)**2)
        # cost = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(tf.transpose(y),tf.transpose(outf)))
        print(cost)
        all_weights = [i for i in weight.values()]
        dw3,db3,dw2,db2,dw1,db1 = tape.gradient(cost,all_weights)
        return {'dw3':dw3, 'db3':db3, 'dw2':dw2, 'db2':db2, 'dw1':dw1, 'db1':db1}


In [92]:
def update_weights(lamb,weight,grads,X,y):
    for i in range(500):
        grads = back(weight,X,y)
        weight['wl3'].assign_sub(lamb * grads['dw3'])
        weight['bl3'].assign_sub(lamb * grads['db3'])
        weight['wl2'].assign_sub(lamb * grads['dw2'])
        weight['bl2'].assign_sub(lamb * grads['db2'])
        weight['wl1'].assign_sub(lamb * grads['dw1'])
        weight['bl1'].assign_sub(lamb * grads['db1'])
        
    return weight



In [91]:
def compute_accuracy(y,pred):
    Y = np.argmax(y, axis = 1)
    Ypred = np.argmax(pred, axis = 1)
    accuracy = (Y == Ypred).sum()/Y.size
    return accuracy

In [99]:
data = pd.read_csv('./iris.data',names=['x1','x2','x3','x4','y'])
for col in ['x1','x2','x3','x4']:
    data[col] = (data[col] - data[col].mean()) / data[col].std()
datax = tf.convert_to_tensor(data[['x1','x2','x3','x4']],dtype=tf.float32)
one_hot = pd.get_dummies(data['y'])
datay = tf.convert_to_tensor(one_hot[['Iris-setosa','Iris-versicolor','Iris-virginica']],dtype=tf.float32)
X = tf.transpose(datax)
Y = tf.transpose(datay)
Wl1,bl1 = initialize_weights(4,450)
Wl2,bl2 = initialize_weights(450,400)
Wl3,bl3 = initialize_weights(400,3)
weights = {'wl3':Wl3,'bl3':bl3,'wl2':Wl2,'bl2':bl2,'wl1':Wl1,'bl1':bl1}

_,pred = forward_propagation(weights,X,Y)
grads = back(weights,X,Y)
new_weight = update_weights(0.2,weights,grads,X,Y)
_,pred = forward_propagation(new_weight,X,Y)


tf.Tensor(0.28514963, shape=(), dtype=float32)
tf.Tensor(0.28514963, shape=(), dtype=float32)
tf.Tensor(0.28002965, shape=(), dtype=float32)
tf.Tensor(0.27422324, shape=(), dtype=float32)
tf.Tensor(0.26386806, shape=(), dtype=float32)
tf.Tensor(0.39387384, shape=(), dtype=float32)
tf.Tensor(0.34134364, shape=(), dtype=float32)
tf.Tensor(0.31966376, shape=(), dtype=float32)
tf.Tensor(0.30100837, shape=(), dtype=float32)
tf.Tensor(0.29720616, shape=(), dtype=float32)
tf.Tensor(0.29626533, shape=(), dtype=float32)
tf.Tensor(0.29394498, shape=(), dtype=float32)
tf.Tensor(0.29252198, shape=(), dtype=float32)
tf.Tensor(0.2913583, shape=(), dtype=float32)
tf.Tensor(0.2896667, shape=(), dtype=float32)
tf.Tensor(0.28834963, shape=(), dtype=float32)
tf.Tensor(0.2871599, shape=(), dtype=float32)
tf.Tensor(0.2860981, shape=(), dtype=float32)
tf.Tensor(0.28565085, shape=(), dtype=float32)
tf.Tensor(0.28531227, shape=(), dtype=float32)
tf.Tensor(0.2849855, shape=(), dtype=float32)
tf.Tensor(0.284641

In [103]:
print(compute_accuracy(tf.transpose(Y[:,100:150]),tf.transpose(pred[:,100:150])))

1.0


In [1]:
(20 + 18.7*3 + 17.7*3 + 20 + 17.25 + 19.69*3 + 16.5 + 20*2)/15

18.801333333333332